In [1]:
from importlib.metadata import version

import torch
print("파이토치 버전:", version("torch"))

파이토치 버전: 2.8.0+cu126


# 연습문제 3.1

In [3]:
inputs = torch.tensor([
    [0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55], # step (x^6)
])

d_in, d_out = 3, 2

In [9]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

  def __init__(self, d_in, d_out):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(d_in, d_out))
    self.W_key = nn.Parameter(torch.rand(d_in, d_out))
    self.W_value = nn.Parameter(torch.rand(d_in, d_out))

  def forward(self, x):
    keys = x @ self.W_key
    queries = x @ self.W_query
    values = x @ self.W_value

    attn_scores = queries @ keys.T # omega
    attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    context_vec = attn_weights @ values
    return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)

In [10]:
class SelfAttention_v2(nn.Module):

  def __init__(self, d_in, d_out, qkv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

  def forward(self, x):
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.T # omega
    attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    context_vec = attn_weights @ values
    return context_vec

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)

In [11]:
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

In [12]:
sa_v1(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

In [13]:
sa_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

# 연습 문제 3.2

In [20]:
class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()
    self.d_out = d_out
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout) # 추가
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # 추가

  def forward(self, x):
    b, num_tokens, d_in = x.shape # b: 배치 차원
    # 입력의 `num_tokens`가 `context_length`를 넘는 경우 마스크 생성에서 오류가 발생
    # 실제로는 forward 메서드에 들어괴 전에 LLM이 입력이 `context_length`를
    # 넘지 않는지 확인하기 때문에 문제가 되지 않는다.
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.transpose(1, 2) # 전치
    attn_scores.masked_fill( # _ 메서드는 인플레이스 연산
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
    )
    attn_weights = self.dropout(attn_weights) # 추가

    context_vec = attn_weights @ values
    return context_vec

class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    self.heads = nn.ModuleList(
        [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
    )
    self.out_proj = nn.Linear(d_out*num_heads, d_out*num_heads)

  def forward(self, x):
    context_vec = torch.cat([head(x) for head in self.heads], dim=-1)
    return self.out_proj(context_vec)

In [22]:
torch.manual_seed(123)

inputs = torch.tensor([
    [0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55], # step (x^6)
])

batch = torch.stack((inputs, inputs), dim=0)

context_length = batch.shape[1]
d_in = 3
d_out = 1

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.2770, 0.3472],
         [0.2767, 0.3471],
         [0.2767, 0.3471],
         [0.2764, 0.3474],
         [0.2767, 0.3474],
         [0.2763, 0.3473]],

        [[0.2770, 0.3472],
         [0.2767, 0.3471],
         [0.2767, 0.3471],
         [0.2764, 0.3474],
         [0.2767, 0.3474],
         [0.2763, 0.3473]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


# 연습 문제 3.3

In [27]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    assert (d_out % num_heads == 0), "d_out은 num_heads로 나누어 떨어져야 합니다"

    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads # 원하는 출력 차원에 맞도록 투영 차원을 낮춘다.

    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.out_proj = nn.Linear(d_out, d_out) # Linear 층을 사용해 헤드의 출력을 결합
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        "mask", torch.triu(torch.ones(context_length, context_length), diagonal=1)
    )

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    # `CausalAttention`과 마찬가지로, 입력의 `num_tokens`가 `context_length`를 넘는 경우 마스크 생성에서 오류가 발생
    # 실제로는 forward 메서드에 들어오기 전에 LLM이 입력이 `context_length`를 넘지 않는지 확인하기 때문에 문제가 되지 않는다.

    keys = self.W_key(x) # 크기: (b, num_tokens, d_out)
    queries = self.W_query(x)
    values = self.W_value(x)

    # `num_heads` 차원을 추가함으로써 암묵적으로 행렬을 분할한다.
    # 그다음 마지막 차원을 `num_heads`에 맞춰 채운다.: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)

    # 전치: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
    keys = keys.transpose(1,2)
    queries = queries.transpose(1,2)
    values = values.transpose(1,2)

    # 코잘 마스크로 스케일드 점곱 어텐션(셀프 어텐션)을 계산
    attn_scores = queries @ keys.transpose(2,3) # 각 헤드에 대해 점곱을 수행

    # 마스크를 불리언 타입으로 만들고 토큰 개수로 마스크를 자른다.
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

    # 마스크를 사용해 어텐션 점수를 채운다.
    attn_scores.masked_fill(mask_bool, -torch.inf)

    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)

    # 크기: (b, num_tokens, num_heads, head_dim)
    context_vec = (attn_weights @ values).transpose(1, 2)

    # 헤드를 결합한다. self.d_out = self.num_heads * self.head_dim
    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec) # 투영

    return context_vec

In [33]:
context_length = 1024
d_in, d_out = 768, 768
num_heads = 12

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)

In [34]:
def count_parameteres(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameteres(mha)

2360064